# Practica 4 (De nou)

## Requeriments

In [34]:
import gensim
from gensim.models import Word2Vec, word2vec
from gensim.models.word2vec import PathLineSentences
from typing import Tuple, List, Optional
import os
import re

# Requisites
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np
from scipy import spatial

# Define the Model
import tensorflow as tf

# Preprocessing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

from scipy.stats import pearsonr

# Model linear regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from collections import defaultdict

from transformers import pipeline, AutoTokenizer
from scipy.special import logit

import spacy
from spacy.tokens import Doc
from spacy.attrs import ID

## Codi per entrenar models word2vec

In [2]:
from datasets import load_dataset

dataset = load_dataset("projecte-aina/catalan_general_crawling")
dataset = dataset['train']

c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for projecte-aina/catalan_general_crawling contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/catalan_general_crawling
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
# Getting the first 100 MB of the dataset
size = 0
for i in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 100000000:  # 100 MB
        print('files per 100MB:', i)
        break

dataset_100MB = dataset.select(list(range(i)))


# Getting the first 500 MB of the dataset
size = 0
for j in range(len(dataset)):
    size += len(dataset[j]['text'])  # corrected from dataset[i]['text']
    if size > 500000000:  # 500 MB
        print('files per 500MB:', j)
        break

dataset_500MB = dataset.select(list(range(j)))

# Getting the first 1 GB of the dataset
size = 0
for g in range(len(dataset)):
    size += len(dataset[g]['text'])  # corrected from dataset[i]['text']
    if size > 1000000000:  # 1 GB
        print('files per 1GB:', g)
        break

dataset_1GB = dataset.select(list(range(g)))

# Getting the whole dataset
dataset_complet = dataset

files per 100MB: 57333
files per 500MB: 213432
files per 1GB: 410702


In [4]:
datasets = {}
datasets['100MB'] = dataset_100MB
datasets['500MB'] = dataset_500MB
datasets['1GB'] = dataset_1GB  
datasets['complet'] = dataset_complet

models = {}

In [5]:
stopwords_catala = [ 
    "a", "abans", "ací", "així", "alguns", "alguna", "algunes", "algú", "alhora", 
    "als", "allò", "aquell", "aquelles", "aquell", "aquells", "baix", 
    "cada", "com", "com a", "eixa", "eixes", "eixí", 
    "eixos", "el", "ella", "elles", "ell", "ells", "en", "endavant", "enfront", 
    "ens", "entre", "he", "hem", "heu", "hi", "ho", 
    "i", "igual", "iguals", "ja", "jo", "l'", "la", "les", "li", "els", "tu", "nosaltres", "vosaltres",
    "de", "del", "dels", "d'un", "d'una", "d'uns", "d'unes", "des", "des de",
    ]

# Funció per netejar i tokenitzar el text
def preprocess_text(text):
    # Neteja el text: treu caràcters no desitjats, converteix a minúscules, etc.
    text = re.sub(r'\W+', ' ', text)  # Substitueix caràcters no alfanumèrics per espais
    text = text.lower()  # Converteix a minúscules
    words = text.split()  # Tokenitza
    # elimina stopwords
    words = [word for word in words if word not in stopwords_catala]
    return words

In [ ]:
# Carrega el fitxer de text i processa'l
def create_corpus(dataset):
    corpus = []
    for line in dataset:
        words = preprocess_text(line['text'])
        if words:  # Assegura't que la línia no està buida
            corpus.append(words)
    return corpus

corpus_dict = {}
for mida in ["100MB", "500MB", "1GB", "complet"]:
    dataset = datasets[mida]
    corpus_dict[f"corpus_{mida}"] = create_corpus(dataset=dataset)
    print(f"Corpus {mida} creat")


In [ ]:
def entrenar_word2vec(corpus, mida_vector=100, finestra=5, sg=1, min_count=10, workers=4, epochs=25):
    
    model = word2vec.Word2Vec(corpus, vector_size=mida_vector, window=finestra, min_count=min_count, workers=workers, epochs=epochs, sg=sg)
    
    return model

# Entrenar models per a cada mida de dataset
for mida in ["500MB", "1GB", "complet"]: # "100MB"
    corpus = corpus_dict[f"corpus_{mida}"]
    model = entrenar_word2vec(corpus)
    models[mida] = model
    model.save(f"model_{mida}.model")


In [8]:
for model in ['100MB', '500MB', '1GB', 'complet']:
    # load model
    models[model] = gensim.models.Word2Vec.load(f"model_{model}.model")
    
models['100MB'] = gensim.models.Word2Vec.load("model_100MB.model")

In [15]:
models['500MB'].wv.most_similar("casa", topn=10)

[('caseta', 0.715048611164093),
 ('família', 0.6787399053573608),
 ('llar', 0.6702232360839844),
 ('pairal', 0.6680922508239746),
 ('acollidora', 0.6579598188400269),
 ('cabana', 0.6460942625999451),
 ('bonica', 0.6448329091072083),
 ('jardí', 0.640557587146759),
 ('habitació', 0.6359531283378601),
 ('granja', 0.6318382024765015)]

## Model de Similitud de Text Semàntic

In [9]:
from datasets import load_dataset

text_semantic = load_dataset("projecte-aina/sts-ca")

c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for projecte-aina/sts-ca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/sts-ca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [10]:
text_semantic['train']['sentence1'][0]

'Atorga per primer cop les mencions Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència Universitària'

### Inicialització

In [11]:
input_pairs = [(e["sentence1"], e["sentence2"], e["label"], ) for e in text_semantic["train"].to_list()]
input_pairs_val = [(e["sentence1"], e["sentence2"], e["label"], ) for e in text_semantic["validation"].to_list()]
input_pairs_test = [(e["sentence1"], e["sentence2"], e["label"], ) for e in text_semantic["test"].to_list()]

In [14]:
all_input_pairs = input_pairs + input_pairs_val + input_pairs_test
# Preprocesamiento de las oraciones y creación del diccionario
sentences_1_preproc = [simple_preprocess(sentence_1) for sentence_1, _, _ in all_input_pairs]
sentences_2_preproc = [simple_preprocess(sentence_2) for _, sentence_2, _ in all_input_pairs]
sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc))
# Versión aplanada para poder entrenar el modelo
sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc
diccionario = Dictionary(sentences_pairs_flattened)

In [15]:
all_input_pairs[0]

('Atorga per primer cop les mencions Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència Universitària',
 'Creen la menció M. Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència universitària',
 3.5)

In [16]:
# Cálculo de los pesos TF-IDF para las oraciones pre-procesadas
corpus = [diccionario.doc2bow(sent) for sent in sentences_pairs_flattened]
modelo_tfidf = TfidfModel(corpus)

In [17]:
def map_tf_idf(sentence_preproc: List[str], dictionary: Dictionary, tf_idf_model: TfidfModel, wv_model) -> Tuple[List[np.ndarray], List[float]]:
    bow = dictionary.doc2bow(sentence_preproc)
    tf_idf = tf_idf_model[bow]
    vectors, weights = [], []
    for word_index, weight in tf_idf:
        word = dictionary.get(word_index)
        if word in wv_model:
            vectors.append(wv_model[word])
            weights.append(weight)
    return vectors, weights

def map_pairs(
        sentence_pairs: List[Tuple[str, str, float]],
        dictionary: Dictionary = None,
        tf_idf_model: TfidfModel = None,
        wv_model: Word2Vec = None,
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        sentence_1_preproc = preprocess_text(sentence_1)
        sentence_2_preproc = preprocess_text(sentence_2)
        # Si usamos TF-IDF
        if tf_idf_model is not None:
            # Cálculo del promedio ponderado por TF-IDF de los word embeddings
            vectors1, weights1 = map_tf_idf(sentence_1_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, wv_model=wv_model,)
            vectors2, weights2 = map_tf_idf(sentence_2_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, wv_model=wv_model,)
            vector1 = np.average(vectors1, weights=weights1, axis=0, )
            vector2 = np.average(vectors2, weights=weights2, axis=0, )
        else:
            # Cálculo del promedio de los word embeddings
            vectors1 = [wv_model[word] for word in sentence_1_preproc if word in wv_model]
            vectors2 = [wv_model[word] for word in sentence_2_preproc if word in wv_model]
            vector1 = np.mean(vectors1, axis=0)
            vector2 = np.mean(vectors2, axis=0)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [18]:
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.array(_x_1), np.array(_x_2)), np.array(_y, dtype=np.float32, )

## Model Baseline

In [20]:
def build_and_compile_model_better(embedding_size: int = 300, learning_rate: float = 1e-3) -> tf.keras.Model:
    # Capa de entrada para los pares de vectores
    input_1 = tf.keras.Input(shape=(embedding_size,))
    input_2 = tf.keras.Input(shape=(embedding_size,))

    # Hidden layer
    first_projection = tf.keras.layers.Dense(
        embedding_size,
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
    )
    projected_1 =  first_projection(input_1)
    projected_2 = first_projection(input_2)
    
    # Compute the cosine distance using a Lambda layer
    def normalized_product(x):
        x1, x2 = x
        x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
        x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
        return x1_normalized * x2_normalized

    output = tf.keras.layers.Lambda(normalized_product)([projected_1, projected_2])
    output = tf.keras.layers.Dropout(0.1)(output)
    output = tf.keras.layers.Dense(
        16,
        activation="relu",
    )(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(
        1,
        activation="sigmoid",
    )(output)
    
    output = tf.keras.layers.Lambda(lambda x: x * 5)(output)
    
    # Define output
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate))
    return model

In [21]:
def model_2(
    input_length: int = 37,
    dictionary_size: int = 1000,
    embedding_size: int = 100,
    learning_rate: float = 0.01,
    pretrained_weights: Optional[np.ndarray] = None,
    trainable: bool = False,
    use_cosine: bool = False,
    l2_regularizer: float = 1e-4,
) -> tf.keras.Model:
    # Inputs
    input_1 = tf.keras.Input((input_length,), dtype=tf.int32)
    input_2 = tf.keras.Input((input_length,), dtype=tf.int32)

    # Embedding Layer
    if pretrained_weights is None:
        embedding_layer = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True, embeddings_initializer='uniform'
        )
    else:
        embedding_layer = tf.keras.layers.Embedding(
            pretrained_weights.shape[0], pretrained_weights.shape[1],  # Ajustar la forma de los embeddings pre-entrenados
            input_length=input_length,
            mask_zero=True,
            embeddings_initializer=tf.keras.initializers.Constant(pretrained_weights),
            trainable=trainable,
        )

    # Embed the inputs
    embedded_1 = embedding_layer(input_1)
    embedded_2 = embedding_layer(input_2)
    # Pass through the embedding layer
    _input_mask_1, _input_mask_2 = tf.not_equal(input_1, 0), tf.not_equal(input_2, 0)
    
    # Attention Mechanism
    attention_mlp = tf.keras.Sequential([
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    # Apply attention to each embedding
    attention_weights_1 = attention_mlp(embedded_1)  
    attention_weights_2 = attention_mlp(embedded_2) 
    # Mask the attention weights
    attention_weights_1 = tf.exp(attention_weights_1) * tf.cast(_input_mask_1[:, :, None], tf.float32)
    attention_weights_2 = tf.exp(attention_weights_2) * tf.cast(_input_mask_2[:, :, None], tf.float32)
    # Normalize attention weights
    attention_weights_1 = attention_weights_1 / tf.reduce_sum(attention_weights_1, axis=1, keepdims=True)
    attention_weights_2 = attention_weights_2 / tf.reduce_sum(attention_weights_2, axis=1, keepdims=True)
    # Compute context vectors
    projected_1 = tf.reduce_sum(embedded_1 * attention_weights_1, axis=1) 
    projected_2 = tf.reduce_sum(embedded_2 * attention_weights_2, axis=1) 
    
    if use_cosine:
        # Compute the cosine distance using a Lambda layer
        def cosine_distance(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return 2.5 * (1.0 + tf.reduce_sum(x1_normalized * x2_normalized, axis=1))
        output = tf.keras.layers.Lambda(cosine_distance)([projected_1, projected_2])
    else:
        # Compute the cosine distance using a Lambda layer
        def normalized_product(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return x1_normalized * x2_normalized
    
        output = tf.keras.layers.Lambda(normalized_product)([projected_1, projected_2])
        output = tf.keras.layers.Dropout(0.1)(output)
        output = tf.keras.layers.Dense(
            16,
            activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)
        )(output)
        output = tf.keras.layers.Dropout(0.2)(output)
        output = tf.keras.layers.Dense(
            1,
            activation="sigmoid",
        )(output)
        
        output = tf.keras.layers.Lambda(lambda x: x * 5)(output)
        
    # Model Definition
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=output)
    model.compile(
        loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
    )
    return model


## Diferents models de Word Embeddings

### One Hot

In [88]:
# Calcula el número de muestras de entrenamiento y validación
num_train_samples = len(input_pairs)
num_val_samples = len(input_pairs_val)
num_test_samples = len(input_pairs_test)

# Combina las muestras de entrenamiento y validación para procesarlas juntas
all_input_pairs = input_pairs + input_pairs_val + input_pairs_test

# Preprocesamiento de todas las oraciones y construcción del vocabulario
preprocessed_sentences = [preprocess_text(pair[0]) + preprocess_text(pair[1]) for pair in all_input_pairs]
vocab = set([word for sentence in preprocessed_sentences for word in sentence])
vocab_size = len(vocab)

# Codificación one-hot de cada oración preprocesada
encoded_sentences = [one_hot(' '.join(sentence), vocab_size) for sentence in preprocessed_sentences]

# Obtener la longitud máxima de las secuencias codificadas
max_seq_length = max(len(seq) for seq in encoded_sentences)

# Rellenar las secuencias codificadas para que todas tengan la misma longitud
padded_encoded_sentences = pad_sequences(encoded_sentences, maxlen=max_seq_length)

# Dividir los pares de entrada nuevamente
train_encoded_pairs = [
    (padded_encoded_sentences[i], padded_encoded_sentences[i + num_train_samples], label)
    for i, (_, _, label) in enumerate(input_pairs)
    if i + num_train_samples < len(padded_encoded_sentences)
]

val_encoded_pairs = [
    (padded_encoded_sentences[i], padded_encoded_sentences[i + num_val_samples], label)
    for i, (_, _, label) in enumerate(input_pairs_val)
    if i + num_val_samples < len(padded_encoded_sentences)
]

test_encoded_pairs = [
    (padded_encoded_sentences[i], padded_encoded_sentences[i + num_test_samples], label)
    for i, (_, _, label) in enumerate(input_pairs_test)
    if i + num_test_samples < len(padded_encoded_sentences)
]

# Entrenar el modelo con los datos codificados en one-hot
model_oneHot = build_and_compile_model_better(embedding_size=max_seq_length)

# Entrenar el modelo
history = model_oneHot.fit(
    x=[np.array([pair[0] for pair in train_encoded_pairs]), np.array([pair[1] for pair in train_encoded_pairs])],
    y=np.array([pair[2] for pair in train_encoded_pairs]),
    validation_data=(
        [np.array([pair[0] for pair in val_encoded_pairs]), np.array([pair[1] for pair in val_encoded_pairs])],
        np.array([pair[2] for pair in val_encoded_pairs]),
    ),
    epochs=50,  # Ajustar según sea necesario
    batch_size=32,  # Ajustar según sea necesario
    verbose=1
)

Epoch 1/500
32/32 [==============================] - 2s 15ms/step - loss: 0.7148 - val_loss: 0.7298
Epoch 2/500
32/32 [==============================] - 0s 6ms/step - loss: 0.7083 - val_loss: 0.7294
Epoch 3/500
32/32 [==============================] - 0s 5ms/step - loss: 0.7061 - val_loss: 0.7304
Epoch 4/500
32/32 [==============================] - 0s 5ms/step - loss: 0.7075 - val_loss: 0.7305
Epoch 5/500
32/32 [==============================] - 0s 5ms/step - loss: 0.7053 - val_loss: 0.7317
Epoch 6/500
32/32 [==============================] - 0s 4ms/step - loss: 0.7042 - val_loss: 0.7343
Epoch 7/500
32/32 [==============================] - 0s 4ms/step - loss: 0.7028 - val_loss: 0.7342
Epoch 8/500
32/32 [==============================] - 0s 4ms/step - loss: 0.7004 - val_loss: 0.7357
Epoch 9/500
32/32 [==============================] - 0s 4ms/step - loss: 0.6974 - val_loss: 0.7382
Epoch 10/500
32/32 [==============================] - 0s 4ms/step - loss: 0.6943 - val_loss: 0.7406
Epoch 11

L'entrenament del model està una mica sobreajustat, ja que la loss del conjunt d'entrenament a la que arriba és considerablement inferior a la loss del conjunt de validació.

In [89]:
# Obtener las predicciones del modelo en el conjunto de prueba
predictions = model_oneHot.predict(x=[np.array([pair[0] for pair in test_encoded_pairs]), np.array([pair[1] for pair in test_encoded_pairs])], verbose=1)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(np.array([pair[2] for pair in test_encoded_pairs]), predictions)

print("Coeficiente de determinación (R^2) en el conjunto de prueba:", r2)

mse = mean_squared_error(np.array([pair[2] for pair in test_encoded_pairs]), predictions)
print("\nError cuadrático medio (MSE):", mse)


16/16 [==============================] - 0s 2ms/step
Coeficiente de determinación (R^2) en el conjunto de prueba: -0.8981389642625837

Error cuadrático medio (MSE): 1.4331130363503939


Com havíem mencionat anteriorment, el model al estar tan sobreajustat no funciona bé amb dades no vistes i per tant el R2_score dona negatiu. Indicant que el model prediu els valors pitjor que predint només amb la mitjana.

### Model Word2Vec

Realitzem el model de regressio amb els embeddings dels models Word2Vec + Mean i + Mean Ponderada.

#### Word2Vec + Mean 

In [31]:
def mapp(word2vec, batch_size: int = 64, num_epochs: int = 64):
    
    # Imprimir los pares de vectores y la puntuación de similitud asociada
    mapped = map_pairs(input_pairs, dictionary=diccionario, wv_model=word2vec.wv)
    
    # Imprimir los pares de vectores y la puntuación de similitud asociada
    mapped_train = map_pairs(input_pairs,  dictionary=diccionario, wv_model=word2vec.wv)
    mapped_val = map_pairs(input_pairs_val, dictionary=diccionario, wv_model=word2vec.wv)
    mapped_test = map_pairs(input_pairs_test, dictionary=diccionario, wv_model=word2vec.wv)

    # Obtener las listas de train y test
    x_train, y_train = pair_list_to_x_y(mapped_train)
    x_val, y_val = pair_list_to_x_y(mapped_val)

    # Preparar los conjuntos de datos de entrenamiento y validación
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

    val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    val_dataset = val_dataset.batch(batch_size)
    
    model = build_and_compile_model_better(embedding_size=word2vec.vector_size)
    
    model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, verbose=0)
    
    
    x_test, y_test = pair_list_to_x_y(mapped_test)
    
    # Baseline
    def compute_pearson_baseline(x_, y_):
        y_pred_baseline = []
        for v1, v2 in zip(*x_):
            d = 1.0 - spatial.distance.cosine(v1, v2)
            y_pred_baseline.append(d)
        # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
        correlation, _ = pearsonr(y_pred_baseline, y_.flatten())
        return correlation
    
    # Imprimir el coeficiente de correlación de Pearson
    print(f"Correlación de Pearson (baseline-train): {compute_pearson_baseline(x_train, y_train)}")
    print(f"Correlación de Pearson (baseline-validation): {compute_pearson_baseline(x_val, y_val)}")
    print(f"Correlación de Pearson (baseline-test): {compute_pearson_baseline(x_test, y_test)}\n")
    
    def compute_pearson(x_, y_):
        # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
        y_pred = model.predict(x_)
        # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
        correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
        return correlation

    # Imprimir el coeficiente de correlación de Pearson
    print(f"Correlación de Pearson (train): {compute_pearson(x_train, y_train)}")
    print(f"Correlación de Pearson (validation): {compute_pearson(x_val, y_val)}")
    print(f"Correlación de Pearson (test): {compute_pearson(x_test, y_test)}\n\n")

for model in ['100MB', '500MB', '1GB', 'complet']:
    print(f"Entrenant model {model}\n")
    mapp(models[model])

Entrenant model 100MB

Correlación de Pearson (baseline-train): 0.34132889471710465
Correlación de Pearson (baseline-validation): 0.2878746083296036
Correlación de Pearson (baseline-test): 0.40315940546719736

65/65 [==============================] - 0s 951us/step
Correlación de Pearson (train): 0.9726040743395692
16/16 [==============================] - 0s 901us/step
Correlación de Pearson (validation): 0.24693219167955727
16/16 [==============================] - 0s 931us/step
Correlación de Pearson (test): 0.3190721192769009


Entrenant model 500MB

Correlación de Pearson (baseline-train): 0.452860546989999
Correlación de Pearson (baseline-validation): 0.4618448540799148
Correlación de Pearson (baseline-test): 0.5399993029883255

65/65 [==============================] - 0s 994us/step
Correlación de Pearson (train): 0.961833713815724
16/16 [==============================] - 0s 1ms/step
Correlación de Pearson (validation): 0.30232934045863735
16/16 [==============================] - 0s

Els valors de les Correlacions de Pearson calculats ens indiquen que el model també està molt sobreajustat, amb valors propers al 1 en tots els models pel conjunt de train i amb valors propers a 0.4 per tots els models en els conjunts de test. El model complet obté els millors resultats pel conjunt de test, però la diferència no és gairebé notòria.  

#### Word2Vec + Mean Ponderada

In [48]:
def mapp(word2vec, batch_size: int = 64, num_epochs: int = 64):
    
    # Imprimir los pares de vectores y la puntuación de similitud asociada
    mapped = map_pairs(input_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionario, wv_model=word2vec.wv)
    
    # Imprimir los pares de vectores y la puntuación de similitud asociada
    mapped_train = map_pairs(input_pairs,  tf_idf_model=modelo_tfidf, dictionary=diccionario, wv_model=word2vec.wv)
    mapped_val = map_pairs(input_pairs_val, tf_idf_model=modelo_tfidf, dictionary=diccionario, wv_model=word2vec.wv)
    mapped_test = map_pairs(input_pairs_test, tf_idf_model=modelo_tfidf, dictionary=diccionario, wv_model=word2vec.wv)

    # Obtener las listas de train y test
    x_train, y_train = pair_list_to_x_y(mapped_train)
    x_val, y_val = pair_list_to_x_y(mapped_val)

    # Preparar los conjuntos de datos de entrenamiento y validación
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

    val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    val_dataset = val_dataset.batch(batch_size)
    
    model = build_and_compile_model_better(embedding_size=word2vec.vector_size)
    
    model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, verbose=0)
    
    
    x_test, y_test = pair_list_to_x_y(mapped_test)
    
    # Baseline
    def compute_pearson_baseline(x_, y_):
        y_pred_baseline = []
        for v1, v2 in zip(*x_):
            d = 1.0 - spatial.distance.cosine(v1, v2)
            y_pred_baseline.append(d)
        # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
        correlation, _ = pearsonr(y_pred_baseline, y_.flatten())
        return correlation
    
    # Imprimir el coeficiente de correlación de Pearson
    print(f"Correlación de Pearson (baseline-train): {compute_pearson_baseline(x_train, y_train)}")
    print(f"Correlación de Pearson (baseline-validation): {compute_pearson_baseline(x_val, y_val)}")
    print(f"Correlación de Pearson (baseline-test): {compute_pearson_baseline(x_test, y_test)}\n")
    
    def compute_pearson(x_, y_):
        # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
        y_pred = model.predict(x_)
        # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
        correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
        return correlation

    # Imprimir el coeficiente de correlación de Pearson
    print(f"Correlación de Pearson (train): {compute_pearson(x_train, y_train)}")
    print(f"Correlación de Pearson (validation): {compute_pearson(x_val, y_val)}")
    print(f"Correlación de Pearson (test): {compute_pearson(x_test, y_test)}\n\n")

for model in ['100MB', '500MB', '1GB', 'complet']:
    print(f"Entrenant model {model}\n")
    mapp(models[model])

Entrenant model 100MB

Correlación de Pearson (baseline-train): 0.3648063022380385
Correlación de Pearson (baseline-validation): 0.27936252945628576
Correlación de Pearson (baseline-test): 0.3922879849557813

65/65 [==============================] - 0s 688us/step
Correlación de Pearson (train): 0.97947417844422
16/16 [==============================] - 0s 730us/step
Correlación de Pearson (validation): 0.2206613558163073
16/16 [==============================] - 0s 738us/step
Correlación de Pearson (test): 0.36146354618584164


Entrenant model 500MB

Correlación de Pearson (baseline-train): 0.46535328131056514
Correlación de Pearson (baseline-validation): 0.4927845302072769
Correlación de Pearson (baseline-test): 0.5245793205439706

65/65 [==============================] - 0s 695us/step
Correlación de Pearson (train): 0.9708647046487451
16/16 [==============================] - 0s 646us/step
Correlación de Pearson (validation): 0.2878802680116447
16/16 [==============================] - 0

Els resultats segueixen estan sobreajustats i els valors de les correlacions de Pearson són gairebé els mateixos per cadascún dels models.

#### Word2Vec + Mean amb model de Linear Regression

Hem volgut veure com es comporta l'entrenament del model word2vec amb una linear Regression, ja que el model que utilitzem ens dona molts mals resultats. Amb aquest experiment volem comprovar que no es tracta de la definició del model que usem, sinó de la dificultat de fer prediccions amb aquesta incrustació de paraules.

In [19]:
# Exemple de preprocessament de text
def obtenir_vectores_mean(text, model_word2vec):
    tokens = text.split()
    vectors = [model_word2vec.wv[word] for word in tokens if word in model_word2vec.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model_word2vec.vector_size)

# Exemple d'entrenament del model de regressió
def entrenar_model_regressio(texts, labels, model_word2vec):
    X = np.array([np.concatenate((obtenir_vectores_mean(text1, model_word2vec), obtenir_vectores_mean(text2, model_word2vec))) 
                  for text1, text2 in texts])
    
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2)
    model = LinearRegression()
    model.fit(X_train, y_train)
    prediccions = model.predict(X_test)
    mse = mean_squared_error(y_test, prediccions)
    return model, mse

# Entrenar el model de regressió utilitzant un dels models Word2Vec
texts =  [(sentence1, sentence2) for sentence1, sentence2, _ in input_pairs] # textos del dataset
labels = [label for _, _, label in input_pairs]  # similituds reals
model_word2vec = Word2Vec.load("model_100MB") # utilitzar el model Word2Vec entrenat

model_regressio, mse = entrenar_model_regressio(texts, labels, model_word2vec)
print(f"MSE del model de regressió: {mse}")


MSE del model de regressió: 0.6825629832324949


Un valor d'MSE de 0.6825629832324949 indica que l'error quadràtic mitjà entre les prediccions del model i els valors reals és relativament alt. Això suggereix que el model de regressió no està fent prediccions molt precises.

In [20]:
# Funció per predir la similitud entre dos textos
def predir_similitud(text1, text2, model_regressio, model_word2vec):
    vector1 = obtenir_vectores_mean(text1, model_word2vec)
    vector2 = obtenir_vectores_mean(text2, model_word2vec)
    vector_concat = np.concatenate((vector1, vector2)).reshape(1, -1)
    prediccio = model_regressio.predict(vector_concat)
    return prediccio[0]

# Exemple de predicció
text1 = "Aquest és un text"
text2 = "Aquest és un altre text"
similitud = predir_similitud(text1, text2, model_regressio, model_word2vec)
print(f"Similitud predita: {similitud}")


Similitud predita: 2.6517136096954346


En aquest exemple hem volgut comprovar que predia el model amb dues frases realment similars per veure quant de malament s'equivoca. Efectivament, la similitud predita hauria de ser més elevada i l'error és bastant gran, predint una valor en el mig de la puntuació màxima per una predicció de gairebé el màxim de la puntuació.

### 3. spaCy

In [ ]:
!python -m spacy download ca_core_news_md

In [ ]:
# Cargar el modelo de spaCy
nlp = spacy.load('ca_core_news_md')

def embeddings_spacy(inputs):
    embeddings = []
    for texto1, texto2, _ in inputs:
        doc1 = nlp(texto1)
        embeddings_texto1 = np.mean([token.vector for token in doc1], axis=0)
        doc2 = nlp(texto2)
        embeddings_texto2 = np.mean([token.vector for token in doc2], axis=0)
        embeddings.append((embeddings_texto1, embeddings_texto2))
    return np.array(embeddings), np.array([label for _, _, label in inputs])

def calculate_max_seq_length(inputs):
    max_length = 0
    for texto1, texto2, _ in inputs:
        max_length = max(max_length, len(nlp(texto1)), len(nlp(texto2)))
    return max_length

def prepare_data(embeddings):
    return np.vstack(embeddings)

In [165]:

train_embeddings, labels_train = embeddings_spacy(input_pairs)
val_embeddings, labels_val = embeddings_spacy(input_pairs_val)
test_embeddings, labels_test = embeddings_spacy(input_pairs_test)

max_seq_length = max(calculate_max_seq_length(input_pairs),
                     calculate_max_seq_length(input_pairs_val),
                     calculate_max_seq_length(input_pairs_test))

dictionary_size = len(nlp.vocab)
embedding_size = nlp.vocab.vectors_length

y_train = labels_train
y_val = labels_val
y_test = labels_test

model = build_and_compile_model_better(
    embedding_size=embedding_size,
)

X_train_1 = prepare_data([pair[0] for pair in train_embeddings])
X_train_2 = prepare_data([pair[1] for pair in train_embeddings])
X_val_1 = prepare_data([pair[0] for pair in val_embeddings])
X_val_2 = prepare_data([pair[1] for pair in val_embeddings])
X_test_1 = prepare_data([pair[0] for pair in test_embeddings])
X_test_2 = prepare_data([pair[1] for pair in test_embeddings])

# Entrenar el modelo
history = model.fit(
    x=[X_train_1, X_train_2],
    y=y_train,
    validation_data=([X_val_1, X_val_2], y_val),
    epochs=500,        # Número de épocas
    batch_size=64,    # Tamaño del lote
    verbose=1         # Modo de verbosidad
)

# Evaluar el rendimiento del modelo
evaluation = model.evaluate(
    x=[X_test_1, X_test_2],
    y=y_test
)

print(f'Evaluación del modelo: {evaluation}')


Epoch 1/500
33/33 [==============================] - 1s 12ms/step - loss: 0.6982 - val_loss: 0.7186
Epoch 2/500
33/33 [==============================] - 0s 7ms/step - loss: 0.6856 - val_loss: 0.7070
Epoch 3/500
33/33 [==============================] - 0s 6ms/step - loss: 0.6711 - val_loss: 0.6999
Epoch 4/500
33/33 [==============================] - 0s 7ms/step - loss: 0.6560 - val_loss: 0.6891
Epoch 5/500
33/33 [==============================] - 0s 7ms/step - loss: 0.6338 - val_loss: 0.6819
Epoch 6/500
33/33 [==============================] - 0s 6ms/step - loss: 0.6155 - val_loss: 0.6769
Epoch 7/500
33/33 [==============================] - 0s 6ms/step - loss: 0.5910 - val_loss: 0.6589
Epoch 8/500
33/33 [==============================] - 0s 6ms/step - loss: 0.5654 - val_loss: 0.6513
Epoch 9/500
33/33 [==============================] - 0s 7ms/step - loss: 0.5384 - val_loss: 0.6505
Epoch 10/500
33/33 [==============================] - 0s 7ms/step - loss: 0.5113 - val_loss: 0.6490
Epoch 11

In [166]:
# Hacer predicciones en el conjunto de prueba
y_pred = model.predict([X_test_1, X_test_2])

# Calcular el MSE
mse = mean_squared_error(labels_test, y_pred)
print(f'Mean Squared Error (MSE) en el conjunto de prueba: {mse}')

# Calcular el R² score
r2 = r2_score(labels_test, y_pred)
print(f'R² Score en el conjunto de prueba: {r2}')

16/16 [==============================] - 0s 1ms/step
Mean Squared Error (MSE) en el conjunto de prueba: 0.7538529665932117
R² Score en el conjunto de prueba: 0.001531872978051907


Els resultats són millors que els obtinguts amb els dos models anteriors, però encara són insuficients per anomenar-lo com un bon model. L'r2_score no és negatiu però és molt pròxim a 0, indicant una baixa precisió del model en dades no vistes.

### 4.RoBERTa

In [ ]:
!python -m spacy download ca_core_news_trf

In [26]:
# Cargar el modelo de RoBERTa para catalán
nlp_roberta = spacy.load('ca_core_news_trf')

# Definir la función para obtener la última capa oculta
def get_last_hidden_state(doc):
    if doc.has_extension("trf_data"):
        return doc._.trf_data.last_hidden_layer_state
    else:
        return None

# Registrar la extensión en el objeto Doc
Doc.set_extension("trf_last_hidden_state", getter=get_last_hidden_state, force=True)


In [27]:
def process_embeddings(embeddings):
    max_length = max(max(emb1.lengths[0], emb2.lengths[0]) for emb1, emb2 in embeddings)
    processed_embeddings = []
    for emb_pair in embeddings:
        emb1, emb2 = emb_pair
        emb1_padded = np.pad(emb1.data, ((0, max_length - emb1.lengths[0]), (0, 0)), mode='constant')
        emb2_padded = np.pad(emb2.data, ((0, max_length - emb2.lengths[0]), (0, 0)), mode='constant')
        processed_embeddings.append((emb1_padded, emb2_padded))
    return processed_embeddings


def embeddings_roberta(inputs, method="CLS"):
    embeddings = []

    for texto1, texto2, label in inputs:
        doc1 = nlp_roberta(texto1)
        doc2 = nlp_roberta(texto2)
        
        if method == "CLS":
            embeddings_texto1 = doc1._.trf_data.last_hidden_layer_state[0]
            embeddings_texto2 = doc2._.trf_data.last_hidden_layer_state[0]
        elif method == "Mean":
            embeddings_texto1 = doc1._.trf_data.last_hidden_layer_state
            embeddings_texto2 = doc2._.trf_data.last_hidden_layer_state
            embeddings_texto1 = np.mean(embeddings_texto1, axis=0)
            embeddings_texto2 = np.mean(embeddings_texto2, axis=0)
        
        embeddings.append((embeddings_texto1, embeddings_texto2))
    
    processed_embeddings = process_embeddings(embeddings)
    
    embeddings_array = np.array(processed_embeddings)
    labels_array = np.array([label for _, _, label in inputs])
    
    return embeddings_array, labels_array


#### CLS

In [28]:
# Obtener los embeddings CLS para cada conjunto de datos
train_embeddings_cls, labels_train_cls = embeddings_roberta(input_pairs, method="CLS")
val_embeddings_cls, labels_val_cls = embeddings_roberta(input_pairs_val, method="CLS")
test_embeddings_cls, labels_test_cls = embeddings_roberta(input_pairs_test, method="CLS")


In [29]:
# Eliminar la tercera dimensión de los embeddings
train_embeddings_squeezed = train_embeddings_cls[:, :, 0, :]
val_embeddings_squeezed = val_embeddings_cls[:, :, 0, :]
test_embeddings_squeezed = test_embeddings_cls[:, :, 0, :]


# Verificar las nuevas formas
print("Forma original de train_embeddings:", train_embeddings_cls.shape)
print("Forma nueva de train_embeddings:", train_embeddings_squeezed.shape)

Forma original de train_embeddings: (2073, 2, 7, 768)
Forma nueva de train_embeddings: (2073, 2, 768)


In [30]:
def prepare_data(embeddings):
    return np.vstack(embeddings)

# Preparar datos para CLS
X_train_1_cls = prepare_data([pair[0] for pair in train_embeddings_squeezed])
X_train_2_cls = prepare_data([pair[1] for pair in train_embeddings_squeezed])
X_val_1_cls = prepare_data([pair[0] for pair in val_embeddings_squeezed])
X_val_2_cls = prepare_data([pair[1] for pair in val_embeddings_squeezed])
X_test_1_cls = prepare_data([pair[0] for pair in test_embeddings_squeezed])
X_test_2_cls = prepare_data([pair[1] for pair in test_embeddings_squeezed])

In [31]:
# Entrenar y evaluar con embeddings CLS
model_cls = build_and_compile_model_better(embedding_size=X_train_1_cls.shape[1], learning_rate=1e-3)
history_cls = model_cls.fit(
    x=[X_train_1_cls, X_train_2_cls],
    y=labels_train_cls,
    validation_data=([X_val_1_cls, X_val_2_cls], labels_val_cls),
    epochs=500,
    batch_size=32,
    verbose=1
)
y_pred_cls = model_cls.predict([X_test_1_cls, X_test_2_cls])
mse_cls = mean_squared_error(labels_test_cls, y_pred_cls)
r2_cls = r2_score(labels_test_cls, y_pred_cls)

print(f'\nMean Squared Error (MSE) con CLS: {mse_cls}')
print(f'R² Score con CLS: {r2_cls}')


Epoch 1/500
65/65 [==============================] - 1s 8ms/step - loss: 0.7006 - val_loss: 0.7230
Epoch 2/500
65/65 [==============================] - 0s 6ms/step - loss: 0.6897 - val_loss: 0.7198
Epoch 3/500
65/65 [==============================] - 0s 7ms/step - loss: 0.6753 - val_loss: 0.7107
Epoch 4/500
65/65 [==============================] - 0s 7ms/step - loss: 0.6571 - val_loss: 0.7057
Epoch 5/500
65/65 [==============================] - 0s 7ms/step - loss: 0.6435 - val_loss: 0.7024
Epoch 6/500
65/65 [==============================] - 0s 7ms/step - loss: 0.6146 - val_loss: 0.7047
Epoch 7/500
65/65 [==============================] - 0s 7ms/step - loss: 0.5955 - val_loss: 0.7231
Epoch 8/500
65/65 [==============================] - 1s 8ms/step - loss: 0.5724 - val_loss: 0.7268
Epoch 9/500
65/65 [==============================] - 0s 7ms/step - loss: 0.5548 - val_loss: 0.7325
Epoch 10/500
65/65 [==============================] - 0s 7ms/step - loss: 0.5334 - val_loss: 0.7431
Epoch 11/

Els resultats segueixent estan molt sobreajustats, i la predicció és també dolenta en dades mai vistes tal com ens indica l'r2_score negatiu.  

#### Mean

In [99]:
# Cargar el modelo de spaCy
nlp_trf = spacy.load('ca_core_news_trf')

def embeddings_spacy_trf(inputs, max_embedding_length):
    embeddings_trf = []
    for texto1, texto2, _ in inputs:
        
        doc1 = nlp_trf(' '.join(texto1))
        doc2 = nlp_trf(' '.join(texto2))
        
        embeddings_doc1 = doc1._.trf_data.last_hidden_layer_state
        embeddings_doc2 = doc2._.trf_data.last_hidden_layer_state
        
        embeddings_np1 = np.array(embeddings_doc1.data)
        embeddings_np2 = np.array(embeddings_doc2.data)
        
        sentence_embedding1 = np.mean(embeddings_np1, axis=1)
        sentence_embedding2 = np.mean(embeddings_np1, axis=1)
        
        if sentence_embedding1.shape[0] < max_embedding_length:
            padding_length = max_embedding_length - sentence_embedding1.shape[0]
            embedding1 = np.pad(sentence_embedding1, (0, padding_length), mode='constant')
            
        else:
            embedding1 = sentence_embedding1[:max_embedding_length]
            
        if sentence_embedding2.shape[0] < max_embedding_length:
            padding_length = max_embedding_length - sentence_embedding2.shape[0]
            embedding2 = np.pad(sentence_embedding2, (0, padding_length), mode='constant')
            
        else:
            embedding2 = sentence_embedding2[:max_embedding_length]
            
        embeddings_trf.append((embedding1, embedding2))
            
    return np.array(embeddings_trf), np.array([label for _, _, label in inputs])

def calculate_max_seq_length(inputs):
    max_length = 0
    for texto1, texto2, _ in inputs:
        max_length = max(max_length, len(nlp(texto1)), len(nlp(texto2)))
    return max_length

def prepare_data(embeddings):
    return np.vstack(embeddings)

In [102]:
max_embedding_length = 50

train_embeddings, labels_train = embeddings_spacy_trf(input_pairs, max_embedding_length)
print('Embeddings train fets')
val_embeddings, labels_val = embeddings_spacy_trf(input_pairs_val, max_embedding_length)
print('Embeddings val fets')
test_embeddings, labels_test = embeddings_spacy_trf(input_pairs_test, max_embedding_length)
print('Embeddings val fets')

max_seq_length = max(calculate_max_seq_length(input_pairs),
                     calculate_max_seq_length(input_pairs_val),
                     calculate_max_seq_length(input_pairs_test))

dictionary_size = len(nlp.vocab)
embedding_size = nlp.vocab.vectors_length

y_train = labels_train
y_val = labels_val
y_test = labels_test

Embeddings train fets
Embeddings val fets
Embeddings val fets


In [104]:
model = build_and_compile_model_better(
    embedding_size=50,
)

X_train_1 = prepare_data([pair[0] for pair in train_embeddings])
X_train_2 = prepare_data([pair[1] for pair in train_embeddings])
X_val_1 = prepare_data([pair[0] for pair in val_embeddings])
X_val_2 = prepare_data([pair[1] for pair in val_embeddings])
X_test_1 = prepare_data([pair[0] for pair in test_embeddings])
X_test_2 = prepare_data([pair[1] for pair in test_embeddings])

# Entrenar el modelo
history = model.fit(
    x=[X_train_1, X_train_2],
    y=y_train,
    validation_data=([X_val_1, X_val_2], y_val),
    epochs=500,        # Número de épocas
    batch_size=64,    # Tamaño del lote
    verbose=1         # Modo de verbosidad
)

# Evaluar el rendimiento del modelo
evaluation = model.evaluate(
    x=[X_test_1, X_test_2],
    y=y_test
)

print(f'Evaluación del modelo: {evaluation}')


Epoch 1/500
33/33 [==============================] - 1s 9ms/step - loss: 0.7043 - val_loss: 0.7281
Epoch 2/500
33/33 [==============================] - 0s 6ms/step - loss: 0.7025 - val_loss: 0.7277
Epoch 3/500
33/33 [==============================] - 0s 5ms/step - loss: 0.7008 - val_loss: 0.7281
Epoch 4/500
33/33 [==============================] - 0s 4ms/step - loss: 0.6992 - val_loss: 0.7269
Epoch 5/500
33/33 [==============================] - 0s 5ms/step - loss: 0.6979 - val_loss: 0.7265
Epoch 6/500
33/33 [==============================] - 0s 6ms/step - loss: 0.6987 - val_loss: 0.7253
Epoch 7/500
33/33 [==============================] - 0s 13ms/step - loss: 0.6954 - val_loss: 0.7269
Epoch 8/500
33/33 [==============================] - 0s 10ms/step - loss: 0.6979 - val_loss: 0.7268
Epoch 9/500
33/33 [==============================] - 0s 5ms/step - loss: 0.6946 - val_loss: 0.7250
Epoch 10/500
33/33 [==============================] - 0s 4ms/step - loss: 0.6950 - val_loss: 0.7268
Epoch 1

In [105]:
# Hacer predicciones en el conjunto de prueba
y_pred = model.predict([X_test_1, X_test_2])

# Calcular el MSE
mse = mean_squared_error(labels_test, y_pred)
print(f'Mean Squared Error (MSE) en el conjunto de prueba: {mse}')

# Calcular el R² score
r2 = r2_score(labels_test, y_pred)
print(f'R² Score en el conjunto de prueba: {r2}')

16/16 [==============================] - 0s 3ms/step
Mean Squared Error (MSE) en el conjunto de prueba: 1.0505541224927244
R² Score en el conjunto de prueba: -0.39144482213932874


### 5. RoBERTa fine-tuned

In [80]:
model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Preprcessament del text_semantic
corpus_semantic = []
semantic_score = []
for line in text_semantic['train']:
    frase1 = preprocess_text(line['sentence1'])
    frase2 = preprocess_text(line['sentence2'])

    corpus_semantic.append((frase1, frase2))
    semantic_score.append(line['label'])
    
corpus_semantic_test = []
semantic_score_test = []
for line in text_semantic['test']:
    frase1 = preprocess_text(line['sentence1'])
    frase2 = preprocess_text(line['sentence2'])

    corpus_semantic_test.append((frase1, frase2))
    semantic_score_test.append(line['label'])

In [81]:
def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

In [85]:
fine_runed_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(fine_runed_corpus_semantic):
    frase1 = ' '.join(frase1)
    frase2 = ' '.join(frase2)
    fine_runed_corpus_semantic[i] = (frase1, frase2)

In [92]:
predicciones = pipe(prepare(fine_runed_corpus_semantic), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predicciones:
    prediction['score'] = logit(prediction['score'])

scores = [pred['score'] for pred in predicciones]

mse = mean_squared_error(semantic_score, scores)
r2 = r2_score(semantic_score, scores)

print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)


Error cuadrático medio (MSE): 0.19488246370881346
Coeficiente de determinación (R^2): 0.7218913654462078


In [ ]:
mse = mean_squared_error(semantic_score_test, predicciones)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(semantic_score_test, predicciones)

print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)

In [87]:
fine_runed_corpus_semantic_test = corpus_semantic_test.copy()
for i, (frase1, frase2) in enumerate(fine_runed_corpus_semantic_test):
    frase1 = ' '.join(frase1)
    frase2 = ' '.join(frase2)
    fine_runed_corpus_semantic_test[i] = (frase1, frase2)

In [91]:
predicciones = pipe(prepare(fine_runed_corpus_semantic_test), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predicciones:
    prediction['score'] = logit(prediction['score'])

scores = [pred['score'] for pred in predicciones]

mse = mean_squared_error(semantic_score_test, scores)
r2 = r2_score(semantic_score_test, scores)

print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)

Error cuadrático medio (MSE): 0.3876424155120614
Coeficiente de determinación (R^2): 0.48657282822706216


Observem que l'error quadràtic és significativament inferior en comparació amb els altres models, fet que indica una millor ajustament del model. A més, destaca que el coeficient de determinació R2 és notablement elevat, suggerint una alta capacitat del model per explicar la variabilitat de les dades observades.

### CONCLUSIONS PRIMERA PART

Els resultats dels diferents tipus d'embeddings mostren una variabilitat significativa en el seu rendiment. L'enfocament One-hot és clarament el menys efectiu, amb un coeficient de determinació (R²) de -0.8981 i un error quadràtic mig (MSE) de 1.4331, indicant que aquest mètode no només és incapaç de capturar la variabilitat de les dades, sinó que fins i tot rendeix pitjor que un model que no aprèn res. Això es deu probablement a la seva simplicitat i falta de capacitat per capturar relacions semàntiques entre paraules.

El mètode Word2Vec, tant en la seva versió amb la mitjana com amb la mitjana ponderada (per exemple, amb TF-IDF), mostra una millora substancial. Entrenant models amb diferents mides de dades (100MB, 500MB, 1GB, i model complet), veiem que la correlació de Pearson en el conjunt de prova augmenta a mesura que creix la mida del model. El millor rendiment es troba en els models entrenats amb 500MB i 1GB de dades, on la correlació de Pearson arriba fins a 0.4240 i 0.4081 respectivament per a la versió no ponderada, i 0.4039 per a la versió ponderada. Això indica que Word2Vec, especialment amb més dades d'entrenament, és capaç de capturar relacions semàntiques de manera més efectiva que One-hot.

spaCy ca_core_news_md presenta un rendiment intermedi, amb un MSE de 0.6864 i un R² de 0.0909. Això indica una certa capacitat per capturar la variabilitat de les dades, però encara amb limitacions significatives en comparació amb els models més avançats.

RoBERTa en la seva versió spaCy/ca_core_news_trf / roberta-base-ca-v2 no mostra un bon rendiment, amb un MSE de 1.0506 i un R² de -0.3914. Aquest resultat és similar al de One-hot, suggerint que aquest model no és adequat per a aquesta tasca específica o que necessita més ajustaments o dades d'entrenament per rendir millor.

Finalment, RoBERTa fine-tuned (roberta-base-ca-v2-cased-sts) demostra ser el més efectiu, amb un MSE de 0.3876 i un R² de 0.4866. Això significa que aquest model no només és capaç de capturar les relacions semàntiques entre les paraules de manera efectiva, sinó que també explica una gran part de la variabilitat de les dades. El seu bon rendiment es deu probablement al fine-tuning específic per a la tasca, el que millora la seva capacitat de generalització i precisió.

En conclusió, el model RoBERTa fine-tuned és el més eficaç per a la tasca, seguit pels models Word2Vec, especialment aquells entrenats amb més dades. Els mètodes One-hot i RoBERTa en la seva versió estàndard són els menys efectius, mostrant una capacitat limitada per capturar relacions semàntiques i explicar la variabilitat de les dades.

## Model amb embeddings entrenables

Definim un nou model a entrenar per poder incorporar la inicialització d'aquests embeddings pre-entrenats o la inicialització uniforme dels embeddings.

In [32]:
def model_2_wv_rnd(
    input_length: int = 37,
    dictionary_size: int = 1000,
    embedding_size: int = 100,
    learning_rate: float = 0.01,
    pretrained_weights: Optional[np.ndarray] = None,
    trainable: bool = False,
    use_cosine: bool = False,
    l2_regularizer: float = 1e-4,
) -> tf.keras.Model:
    # Inputs
    input_1 = tf.keras.Input((input_length,), dtype=tf.int32)
    input_2 = tf.keras.Input((input_length,), dtype=tf.int32)

    # Embedding Layer
    if pretrained_weights is None:
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True, embeddings_initializer='uniform'
        )
    else:
        # Obtener la matriz de embeddings del modelo Word2Vec
        word_vectors = pretrained_weights.wv
        embedding_matrix = np.zeros((len(word_vectors), embedding_size))
        for i in range(len(word_vectors)):
            embedding_matrix[i] = word_vectors[word_vectors.index_to_key[i]]
            
        dictionary_size = embedding_matrix.shape[0]
        embedding_size = embedding_matrix.shape[1]
        initializer = tf.keras.initializers.Constant(embedding_matrix)
        embedding = tf.keras.layers.Embedding(
            dictionary_size,
            embedding_size,
            input_length=input_length,
            mask_zero=True,
            embeddings_initializer=initializer,
            trainable=trainable,
        )

    # Embed the inputs
    embedded_1 = embedding(input_1)
    embedded_2 = embedding(input_2)
    # Pass through the embedding layer
    _input_mask_1, _input_mask_2 = tf.not_equal(input_1, 0), tf.not_equal(input_2, 0)
    
    # Attention Mechanism
    attention_mlp = tf.keras.Sequential([
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    # Apply attention to each embedding
    attention_weights_1 = attention_mlp(embedded_1)  
    attention_weights_2 = attention_mlp(embedded_2) 
    # Mask the attention weights
    attention_weights_1 = tf.exp(attention_weights_1) * tf.cast(_input_mask_1[:, :, None], tf.float32)
    attention_weights_2 = tf.exp(attention_weights_2) * tf.cast(_input_mask_2[:, :, None], tf.float32)
    # Normalize attention weights
    attention_weights_1 = attention_weights_1 / tf.reduce_sum(attention_weights_1, axis=1, keepdims=True)
    attention_weights_2 = attention_weights_2 / tf.reduce_sum(attention_weights_2, axis=1, keepdims=True)
    # Compute context vectors
    projected_1 = tf.reduce_sum(embedded_1 * attention_weights_1, axis=1) 
    projected_2 = tf.reduce_sum(embedded_2 * attention_weights_2, axis=1) 
    
    if use_cosine:
        # Compute the cosine distance using a Lambda layer
        def cosine_distance(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return 2.5 * (1.0 + tf.reduce_sum(x1_normalized * x2_normalized, axis=1))
        output = tf.keras.layers.Lambda(cosine_distance)([projected_1, projected_2])
    else:
         # Compute the cosine distance using a Lambda layer
        def normalized_product(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return x1_normalized * x2_normalized
    
        output = tf.keras.layers.Lambda(normalized_product)([projected_1, projected_2])
        output = tf.keras.layers.Dropout(0.1)(output)
        output = tf.keras.layers.Dense(
            16,
            activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)
        )(output)
        output = tf.keras.layers.Dropout(0.2)(output)
        output = tf.keras.layers.Dense(
            1,
            activation="sigmoid",
        )(output)
        
        output = tf.keras.layers.Lambda(lambda x: x * 5)(output)
        
    # Model Definition
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=output)
    model.compile(
        loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
    )
    return model

baseline_model = model_2(use_cosine = True)
baseline_model.compile()
baseline_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 37)]                 0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 37)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 37, 100)              100000    ['input_3[0][0]',             
                                                                     'input_4[0][0]']             
                                                                                                  
 tf.math.not_equal (TFOpLam  (None, 37)                   0         ['input_3[0][0]']       

In [33]:
# Tokenitzar i preparar les seqüències d'entrada per als models
def tokenitzar(text):
    return text.lower().split()

def construir_diccionari(texts):
    word_to_index = defaultdict(lambda: len(word_to_index))
    for text in texts:
        tokens = tokenitzar(text)
        for token in tokens:
            word_to_index[token]
    return dict(word_to_index)

def text_to_sequence(text, word_to_index):
    tokens = tokenitzar(text)
    return [word_to_index[token] for token in tokens]

Realitzem les particions amb les seqüències d'entrada esperades pel model

In [35]:
#################
##### TRAIN #####
#################

# Suposem que tens els teus texts i labels
texts = [(sentence1, sentence2) for sentence1, sentence2, _ in input_pairs]
labels = [label for _, _, label in input_pairs]

# Obtenim tots els textos del dataset
all_texts = [text for pair in input_pairs for text in pair[:2]]
word_to_index = construir_diccionari(all_texts)

# Convertir tots els textos a seqüències d'índexs
input_sequences = [(text_to_sequence(pair[0], word_to_index), text_to_sequence(pair[1], word_to_index)) for pair in input_pairs]

# max length of the sequences
max_len = max(max(len(seq[0]), len(seq[1])) for seq in input_sequences) 

# Padding de les seqüències d'índexs
input_1_sequences = pad_sequences([seq[0] for seq in input_sequences], maxlen=max_len, padding='post')
input_2_sequences = pad_sequences([seq[1] for seq in input_sequences], maxlen=max_len, padding='post')

# Convertir les seqüències i labels a arrays numpy
input_1_sequences = np.array(input_1_sequences)
input_2_sequences = np.array(input_2_sequences)
labels = np.array(labels)

# Ajustar la mida del diccionari basat en el nombre de paraules úniques
dictionary_size = len(word_to_index)



##################
###### TEST ######
##################

texts_test = [(sentence1, sentence2) for sentence1, sentence2, _ in input_pairs_test]
labels_test = [label for _, _, label in input_pairs_test]

# Obtenim tots els textos del dataset
all_texts_test = [text for pair in input_pairs_test for text in pair[:2]]
word_to_index_test = construir_diccionari(all_texts_test)

# Convertir tots els textos a seqüències d'índexs
input_sequences_test = [(text_to_sequence(pair[0], word_to_index_test), text_to_sequence(pair[1], word_to_index_test)) for pair in input_pairs_test]

# max length of the sequences
max_len_test = max(max(len(seq[0]), len(seq[1])) for seq in input_sequences_test) 

# Padding de les seqüències d'índexs
input_1_sequences_test = pad_sequences([seq[0] for seq in input_sequences_test], maxlen=max_len, padding='post')
input_2_sequences_test = pad_sequences([seq[1] for seq in input_sequences_test], maxlen=max_len, padding='post')

# Convertir les seqüències i labels a arrays numpy
input_1_sequences_test = np.array(input_1_sequences_test)
input_2_sequences_test= np.array(input_2_sequences_test)
labels_test = np.array(labels_test)


######################
##### VALIDATION #####
######################

# Suposem que tens els teus texts i labels
texts_val = [(sentence1, sentence2) for sentence1, sentence2, _ in input_pairs_val]
labels_val = [label for _, _, label in input_pairs_val]

# Obtenim tots els textos del dataset
all_texts_val = [text for pair in input_pairs_val for text in pair[:2]]
word_to_index_val = construir_diccionari(all_texts_val)

# Convertir tots els textos a seqüències d'índexs
input_sequences_val = [(text_to_sequence(pair[0], word_to_index_val), text_to_sequence(pair[1], word_to_index_val)) for pair in input_pairs_val]

# max length of the sequences
max_len_val = max(max(len(seq[0]), len(seq[1])) for seq in input_sequences_val) 

# Padding de les seqüències d'índexs
input_1_sequences_val = pad_sequences([seq[0] for seq in input_sequences_val], maxlen=max_len, padding='post')
input_2_sequences_val = pad_sequences([seq[1] for seq in input_sequences_val], maxlen=max_len, padding='post')


# Convertir les seqüències i labels a arrays numpy
input_1_sequences_val = np.array(input_1_sequences_val)
input_2_sequences_val= np.array(input_2_sequences_val)
labels_val = np.array(labels_val)

In [36]:
batch_size = 64

# Crear Dataset de TensorFlow per a les dades d'entrenament
train_dataset = tf.data.Dataset.from_tensor_slices(((input_1_sequences, input_2_sequences), labels))
train_dataset = train_dataset.shuffle(buffer_size=len(labels)).batch(batch_size)

# Crear Dataset de TensorFlow per a les dades de validació
val_dataset = tf.data.Dataset.from_tensor_slices(((input_1_sequences_val, input_2_sequences_val), labels_val))
val_dataset = val_dataset.batch(batch_size)

# Crear Dataset de TensorFlow per a les dades de test
test_dataset = tf.data.Dataset.from_tensor_slices(((input_1_sequences_test, input_2_sequences_test), labels_test))
test_dataset = test_dataset.batch(batch_size)

### Resultats

#### Word2Vec

In [37]:
# Inicialitzem diferents models de baseline per cada model de Word2Vec

word2vec_embedding_models = {}
for model in models:
    word2vec_embedding_models[f"{model}"] = model_2_wv_rnd(pretrained_weights=models[model], trainable=True, use_cosine=True)

In [38]:
def entrenament_wv(model: gensim.models.word2vec, input_1_test, input_2_test, labels_test, batch_size: int = 64):
    
    model.fit([input_1_sequences, input_2_sequences], labels, epochs=50, batch_size = batch_size, 
              validation_data=([input_1_sequences_val, input_2_sequences_val], labels_val), verbose=0)

    # Evaluate en el train
    mse_word2vec = model.evaluate(train_dataset)
    print(f"MSE del model amb Word2Vec Embeddings en train: {mse_word2vec}")

    # Prediccions del test
    prediccions_word2vec = model.predict([input_1_test, input_2_test])

    # MSE
    mse_word2vec = mean_squared_error(labels_test, prediccions_word2vec)
    
    print('\n\n')
    print(f"MSE del model amb Word2Vec Embeddings en test: {mse_word2vec}")
    print(f"R^2 del model amb Word2Vec Embeddings en test: {r2_score(labels_test, prediccions_word2vec)}\n")
    
for word2vec_model in word2vec_embedding_models:
    model = word2vec_embedding_models[word2vec_model]
    print(f"Entrenament embeddings pel model {word2vec_model}")
    datasets = entrenament_wv(model, input_1_sequences_test, input_2_sequences_test, labels_test, batch_size=64)
    
    

Entrenament embeddings pel model 100MB
33/33 [==============================] - 0s 1ms/step - loss: 0.0438
MSE del model amb Word2Vec Embeddings en train: 0.04378463700413704
16/16 [==============================] - 0s 1ms/step



MSE del model amb Word2Vec Embeddings en test: 2.1865111298055795
R^2 del model amb Word2Vec Embeddings en test: -1.8960046179239636

Entrenament embeddings pel model 500MB
33/33 [==============================] - 0s 2ms/step - loss: 0.0953
MSE del model amb Word2Vec Embeddings en train: 0.09532838314771652
16/16 [==============================] - 0s 1ms/step



MSE del model amb Word2Vec Embeddings en test: 2.5535762966634046
R^2 del model amb Word2Vec Embeddings en test: -2.3821775002883054

Entrenament embeddings pel model 1GB
33/33 [==============================] - 0s 1ms/step - loss: 0.1083
MSE del model amb Word2Vec Embeddings en train: 0.10834211111068726
16/16 [==============================] - 0s 1ms/step



MSE del model amb Word2Vec Embeddings en 

#### Random Init

In [41]:
model_random = model_2_wv_rnd(dictionary_size=dictionary_size, use_cosine=True)

model_random.fit([input_1_sequences, input_2_sequences], labels, epochs=50, batch_size = batch_size, 
              validation_data=([input_1_sequences_val, input_2_sequences_val], labels_val), verbose=1)

# Evaluate en el train
mse_word2vec = model.evaluate(train_dataset)
print(f"MSE del model amb Word2Vec Embeddings en train: {mse_word2vec}")

# Prediccions del test
prediccions_word2vec = model.predict([input_1_sequences_test, input_1_sequences_test])

# MSE
mse_word2vec = mean_squared_error(labels_test, prediccions_word2vec)

print('\n\n')
print(f"MSE del model amb Word2Vec Embeddings en test: {mse_word2vec}")
print(f"R^2 del model amb Word2Vec Embeddings en test: {r2_score(labels_test, prediccions_word2vec)}")

Epoch 1/50
33/33 [==============================] - 2s 16ms/step - loss: 2.0318 - val_loss: 2.4177
Epoch 2/50
33/33 [==============================] - 0s 12ms/step - loss: 0.8152 - val_loss: 2.5918
Epoch 3/50
33/33 [==============================] - 0s 11ms/step - loss: 0.7425 - val_loss: 2.6697
Epoch 4/50
33/33 [==============================] - 0s 11ms/step - loss: 0.5150 - val_loss: 2.5519
Epoch 5/50
33/33 [==============================] - 0s 12ms/step - loss: 0.4590 - val_loss: 2.5615
Epoch 6/50
33/33 [==============================] - 0s 11ms/step - loss: 0.3494 - val_loss: 2.6073
Epoch 7/50
33/33 [==============================] - 0s 12ms/step - loss: 0.2471 - val_loss: 2.6818
Epoch 8/50
33/33 [==============================] - 0s 11ms/step - loss: 0.1835 - val_loss: 2.7290
Epoch 9/50
33/33 [==============================] - 0s 12ms/step - loss: 0.1501 - val_loss: 2.6524
Epoch 10/50
33/33 [==============================] - 0s 11ms/step - loss: 0.1250 - val_loss: 2.5976
Epoch 11/

### Anàlisis Resultats models embeddings entrenables

En aquesta anàlisi compararem els resultats obtinguts per dos tipus d'embeddings: Random Embeddings (uniforme) i Word2Vec Embeddings, per avaluar quin dels dos és més efectiu per a la tasca de predicció de similitut de frases.

Els resultats mostren que els valors de MSE en el conjunt de test són més alts amb els Random Embeddings i els valors de R^2 són més negatius en comparació amb els Word2Vec Embeddings. Això suggereix que els Random Embeddings presenten un rendiment més baix i menys precisió en les prediccions enfront dels Word2Vec Embeddings, que mostren una millor adaptació i generalització en aquesta tasca particular. Tot i així, ambdós models no servirien per fer una tasca de predicció de similitut entre frases adequadament.

## CONCLUSIONS

Finalment, observem que gairebé tots els models estan sobreajustats i prediuen amb molt mala precisió. Tanmateix, el model de RoBERTa-fine-tuned és el millor model amb diferència, l'r2_score és l'únic el qual està per sobre del 0.5.

En resum, els resultats indiquen que en general, hi ha una variabilitat significativa en el rendiment dels diferents tipus d'embeddings analitzats, des dels One-hot fins als més avançats com Word2Vec i RoBERTa. Entre aquests, RoBERTa fine-tuned emergeix com el més efectiu, amb la millor capacitat per capturar relacions semàntiques i explicar la variabilitat de les dades.

En una comparació específica entre Random Embeddings i Word2Vec Embeddings, es constata que, tot i que els dos models poden mostrar un bon ajustament durant l'entrenament, és clar que estan sobreajustant molt. Tot i això, els models Word2Vec amb emeddings entrenables tenen una millor generalització i rendiment en el conjunt de test en aquest context particular.